In [ ]:
import numpy as np
import pandas as pd
import networkx.drawing.nx_pydot as gl
import networkx as nx
import matplotlib.pyplot as plt
from pprint import pprint

states = ['O1', 'O2', 'O3']
pi_obs = [0.25, 0.4, 0.35]

state_space = pd.Series(pi_obs, index=states, name='states')
print(state_space)
print(state_space.sum())

q_df = pd.DataFrame(columns=states, index=states)
q_df.loc[states[0]] = [0.4, 0.2, 0.4]
q_df.loc[states[1]] = [0.45, 0.45, 0.1]
q_df.loc[states[2]] = [0.45, 0.25, 0.3]
print(q_df)

q = q_df.values
print(q, q.shape)
print('\n')
print(q_df.sum(axis=1))

def _get_markov_edges(Q):
    edges = {}
    for col in Q.columns:
        for idx in Q.index:
            edges[(idx, col)] = Q.loc[idx, col]
    return edges

edges_wts = _get_markov_edges(q_df)
pprint(edges_wts)

G = nx.MultiDiGraph()
G.add_nodes_from(states)
print('Nodes:\n')
print(G.nodes())
print('\n')

for k, v in edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)

print('Edges:')
pprint(G.edges(data=True))

pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='dot')
nx.draw_networkx(G, pos)
plt.show()

hidden_states = ['S1', 'S2']
pi = [0.5, 0.5]
print('\n')
state_space = pd.Series(pi, index=hidden_states, name='states')
print(state_space)
print('\n')
print(state_space.sum())

a_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
a_df.loc[hidden_states[0]] = [0.7, 0.3]
a_df.loc[hidden_states[1]] = [0.4, 0.6]
print(a_df)

a = a_df.values
print('\n')
print(a)
print(a.shape)
print('\n')
print(a_df.sum(axis=1))

observable_states = states
b_df = pd.DataFrame(columns=observable_states, index=hidden_states)
b_df.loc[hidden_states[0]] = [0.2, 0.6, 0.2]
b_df.loc[hidden_states[1]] = [0.4, 0.1, 0.5]
print(b_df)

b = b_df.values
print('\n')
print(b)
print(b.shape)
print('\n')
print(b_df.sum(axis=1))

hide_edges_wts = _get_markov_edges(a_df)
pprint(hide_edges_wts)

emit_edges_wts = _get_markov_edges(b_df)
pprint(emit_edges_wts)

G = nx.MultiDiGraph()
G.add_nodes_from(hidden_states)
print('Nodes:\n')
print(G.nodes())
print('\n')

for (origin, destination), weight in hide_edges_wts.items():
    G.add_edge(origin, destination, weight=weight, label=weight)

pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='neato')
nx.draw_networkx(G, pos, with_labels=True, arrows=True)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.show()

obs_seq = ['O2', 'O1', 'O3']

obs_map = {state: idx for idx, state in enumerate(observable_states)}
obs_idx_seq = [obs_map[obs] for obs in obs_seq]

n_states = len(hidden_states)
T = len(obs_seq)

delta = np.zeros((T, n_states))
psi = np.zeros((T, n_states), dtype=int)

for i in range(n_states):
    delta[0, i] = pi[i] * b[i, obs_idx_seq[0]]

for t in range(1, T):
    for j in range(n_states):
        max_prob = -1
        for i in range(n_states):
            prob = delta[t-1, i] * a[i, j] * b[j, obs_idx_seq[t]]
            if prob > max_prob:
                max_prob = prob
                psi[t, j] = i
        delta[t, j] = max_prob

path = np.zeros(T, dtype=int)
path[T-1] = np.argmax(delta[T-1])
for t in range(T-2, -1, -1):
    path[t] = psi[t+1, path[t+1]]

state_map = {0: 'S1', 1: 'S2'}
state_path = [state_map[v] for v in path]

result = pd.DataFrame({
    'Observation': obs_seq,
    'Best_Path': state_path
})
print(result)
